In [100]:
import numpy as np
import pandas as pd
import csv
import multiprocessing
import re

In [144]:
#upload Data CSV yang digunakan
from google.colab import files
uploaded = files.upload()

Saving dfTransjakarta.csv to dfTransjakarta (4).csv


In [102]:
#dataset trans Jakarta untuk top 100
df = pd.read_csv('dfTransjakarta.csv', encoding='utf-8', sep=',')
df.head(100)

,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,...,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount
0,EIIW227B8L34VB,180062659848800,emoney,Bajragin Usada,M,2008,5,Matraman Baru - Ancol,1.0,P00142,...,106.84402,7,2023-04-03 05:21:44,P00253,Tegalan,-6.203101,106.85715,12.0,2023-04-03 06:00:53,3500.0
1,LGXO740D2N47GZ,4885331907664776,dki,Gandi Widodo,F,1997,6C,Stasiun Tebet - Karet via Patra Kuningan,0.0,B01963P,...,106.83302,13,2023-04-03 05:42:44,B03307P,Sampoerna Strategic,-6.217152,106.81892,21.0,2023-04-03 06:40:01,3500.0
2,DJWR385V2U57TO,4996225095064169,dki,Emong Wastuti,F,1992,R1A,Pantai Maju - Kota,0.0,B00499P,...,106.81435,38,2023-04-03 05:59:06,B04962P,Simpang Kunir Kemukus,-6.133731,106.81475,39.0,2023-04-03 06:50:55,3500.0
3,JTUZ800U7C86EH,639099174703,flazz,Surya Wacana,F,1978,11D,Pulo Gebang - Pulo Gadung 2 via PIK,0.0,B05587P,...,106.93526,23,2023-04-03 05:44:51,B03090P,Raya Penggilingan,-6.183068,106.93194,29.0,2023-04-03 06:28:16,3500.0
4,VMLO535V7F95NJ,570928206772,flazz,Embuh Mardhiyah,M,1982,12,Tanjung Priok - Pluit,0.0,P00239,...,106.88900,5,2023-04-03 06:17:35,P00098,Kali Besar Barat,-6.135355,106.81143,15.0,2023-04-03 06:57:03,3500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,KNSE319Q1P35CC,502072062834,flazz,Kalim Namaga,M,1971,9D,Pasar Minggu - Tanah Abang,1.0,B05716P,...,106.84455,1,2023-04-03 06:25:22,P00256,MH Thamrin,-6.188149,106.82299,37.0,2023-04-03 07:11:01,3500.0
96,RMJF585F2W51BQ,30385017224303,bni,"dr. Vega Rahimah, S.Pt",M,2003,JAK.117,Tanjung Priok - Tanah Merdeka,0.0,B04040P,...,106.89568,7,2023-04-03 06:23:23,B04863P,Simpang Jln. Kramat Jaya 1,-6.108656,106.91734,25.0,2023-04-03 06:55:01,0.0
97,KFYK655N1H73XS,6590854673757012,dki,Bajragin Najmudin,M,1986,3H,Jelambar - Kota,0.0,P00087,...,106.78651,0,2023-04-03 06:03:19,P00138,Museum Fatahillah,-6.133930,106.81209,9.0,2023-04-03 06:35:04,NaN
98,WTBE175B2A58NC,4963710913075879668,brizzi,Ophelia Firmansyah,M,2000,14,Jakarta International Stadium - Senen,0.0,B04647P,...,106.84024,20,2023-04-03 06:00:02,B05725P,Term. Senen,-6.173560,106.84213,21.0,2023-04-03 06:54:13,3500.0


In [143]:
# Menampilkan Jenis data type dari data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37900 entries, 0 to 37899
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   transID                         37900 non-null  object        
 1   payCardID                       37900 non-null  int64         
 2   payCardBank                     37900 non-null  object        
 3   payCardName                     37900 non-null  object        
 4   payCardSex                      37900 non-null  object        
 5   payCardBirthDate                37900 non-null  int64         
 6   corridorID                      36643 non-null  object        
 7   corridorName                    35970 non-null  object        
 8   direction                       37900 non-null  float64       
 9   tapInStops                      36687 non-null  object        
 10  tapInStopsName                  37900 non-null  object        
 11  ta

In [104]:
# menampilkan data corridor dari csv
df.corridorName

,corridorName
0,Matraman Baru - Ancol
1,Stasiun Tebet - Karet via Patra Kuningan
2,Pantai Maju - Kota
3,Pulo Gebang - Pulo Gadung 2 via PIK
4,Tanjung Priok - Pluit
...,...
37895,Ragunan - MH Thamrin via Semanggi
37896,Pinang Ranti - Pramuka
37897,Cibubur - Balai Kota
37898,Tanah Abang - Jembatan Lima


In [145]:
df.tapOutStopsName

,tapOutStopsName
0,Tegalan
1,Sampoerna Strategic
2,Simpang Kunir Kemukus
3,Raya Penggilingan
4,Kali Besar Barat
...,...
37895,SMK 57
37896,Pinang Ranti
37897,Buperta Cibubur
37898,JPO Blok G


In [147]:
# menamilkan data yang digunakan sebagai acuan jarak dalam konteks ini Pantai Maju dan Simpang Kunir Kemuku
filter_df = df[
    (df['corridorName'].str.contains('Pantai Maju')) &
    (df['tapOutStopsName'].str.contains('Simpang Kunir Kemuku'))
]

In [148]:
filter_df[['corridorName','tapOutStopsName']].head(100)

,corridorName,tapOutStopsName
2,Pantai Maju - Kota,Simpang Kunir Kemukus
602,Pantai Maju - Kota,Simpang Kunir Kemukus
1202,Pantai Maju - Kota,Simpang Kunir Kemukus
1802,Pantai Maju - Kota,Simpang Kunir Kemukus
2402,Pantai Maju - Kota,Simpang Kunir Kemukus
3002,Pantai Maju - Kota,Simpang Kunir Kemukus
4802,Pantai Maju - Kota,Simpang Kunir Kemukus
5402,Pantai Maju - Kota,Simpang Kunir Kemukus
6002,Pantai Maju - Kota,Simpang Kunir Kemukus
6602,Pantai Maju - Kota,Simpang Kunir Kemukus


In [149]:
filter_df[['tapInTime', 'corridorName', 'tapOutStopsName']].head(100)

,tapInTime,corridorName,tapOutStopsName
2,2023-04-03 05:59:06,Pantai Maju - Kota,Simpang Kunir Kemukus
602,2023-04-04 06:25:26,Pantai Maju - Kota,Simpang Kunir Kemukus
1202,2023-04-05 05:05:13,Pantai Maju - Kota,Simpang Kunir Kemukus
1802,2023-04-06 06:07:45,Pantai Maju - Kota,Simpang Kunir Kemukus
2402,2023-04-07 06:35:01,Pantai Maju - Kota,Simpang Kunir Kemukus
3002,2023-04-10 06:06:32,Pantai Maju - Kota,Simpang Kunir Kemukus
4802,2023-04-13 05:59:43,Pantai Maju - Kota,Simpang Kunir Kemukus
5402,2023-04-14 06:02:04,Pantai Maju - Kota,Simpang Kunir Kemukus
6002,2023-04-17 06:01:24,Pantai Maju - Kota,Simpang Kunir Kemukus
6602,2023-04-18 06:23:28,Pantai Maju - Kota,Simpang Kunir Kemukus


In [150]:
filter_df[['tapOutTime', 'corridorName', 'tapOutStopsName']].head(100)

,tapOutTime,corridorName,tapOutStopsName
2,2023-04-03 06:50:55,Pantai Maju - Kota,Simpang Kunir Kemukus
602,2023-04-04 06:53:35,Pantai Maju - Kota,Simpang Kunir Kemukus
1202,2023-04-05 05:37:14,Pantai Maju - Kota,Simpang Kunir Kemukus
1802,2023-04-06 06:39:02,Pantai Maju - Kota,Simpang Kunir Kemukus
2402,2023-04-07 07:26:58,Pantai Maju - Kota,Simpang Kunir Kemukus
3002,2023-04-10 06:52:10,Pantai Maju - Kota,Simpang Kunir Kemukus
4802,2023-04-13 06:48:07,Pantai Maju - Kota,Simpang Kunir Kemukus
5402,2023-04-14 06:59:26,Pantai Maju - Kota,Simpang Kunir Kemukus
6002,2023-04-17 06:22:40,Pantai Maju - Kota,Simpang Kunir Kemukus
6602,2023-04-18 06:45:44,Pantai Maju - Kota,Simpang Kunir Kemukus


In [151]:
df['tapInTime'] = pd.to_datetime(df['tapInTime'])
df['tapOutTime'] = pd.to_datetime(df['tapOutTime'])

In [152]:
waktu_tap_in = df['tapInTime'].dt.time
waktu_tap_out = df['tapOutTime'].dt.time

In [153]:
#print(waktu_tap_in[[2,602,1202,1802,2402]]) # menampilkan 5 baris jam tap-in yang berkaitan dengan Pantai Maju - Kota
#print(waktu_tap_out[[2,602,1202,1802,2402]])  # menampilkan 5 baris jam tap-out yang berkaitan dengan Pantai Maju - Kota
print(waktu_tap_in.head()) # menampilkan 5 baris pertama
print(waktu_tap_out.head()) # menampilkan 5 baris pertama

0    05:21:44
1    05:42:44
2    05:59:06
3    05:44:51
4    06:17:35
Name: tapInTime, dtype: object
0    06:00:53
1    06:40:01
2    06:50:55
3    06:28:16
4    06:57:03
Name: tapOutTime, dtype: object


In [154]:
def waktu_per_detik(t):
  return t.hour * 3600 + t.minute * 60 + t.second

df['waktu_per_detik_tap_in'] = df['tapInTime'].apply(waktu_per_detik)
df['waktu_per_detik_tap_out'] = df['tapOutTime'].apply(waktu_per_detik)\

df['waktu_perbedaan_tap_in_tap_out'] = df['waktu_per_detik_tap_out'] - df['waktu_per_detik_tap_in']

In [155]:
df[['waktu_perbedaan_tap_in_tap_out','waktu_per_detik_tap_out','waktu_per_detik_tap_in']]

,waktu_perbedaan_tap_in_tap_out,waktu_per_detik_tap_out,waktu_per_detik_tap_in
0,2349.0,21653.0,19304
1,3437.0,24001.0,20564
2,3109.0,24655.0,21546
3,2605.0,23296.0,20691
4,2368.0,25023.0,22655
...,...,...,...
37895,5832.0,71749.0,65917
37896,2151.0,80902.0,78751
37897,10658.0,48445.0,37787
37898,1704.0,73670.0,71966


In [156]:
df[['waktu_perbedaan_tap_in_tap_out']]

,waktu_perbedaan_tap_in_tap_out
0,2349.0
1,3437.0
2,3109.0
3,2605.0
4,2368.0
...,...
37895,5832.0
37896,2151.0
37897,10658.0
37898,1704.0
